In [1]:
%config InlineBackend.figure_formats = ['svg']
from ipywidgets import *
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.stats as stats
from matplotlib.lines import Line2D
import matplotlib.patches as mpatches
plt.rcParams['axes.grid'] = True

In [29]:
def describe(data):
    df = pd.DataFrame(data)
    print(df.describe())

def histogram(data, a=1, b=0):
    data = data*a+b
    plt.hist(x=data, bins='auto', color='#521422', alpha=0.7, rwidth=0.85)
    plt.grid(axis='y', alpha=0.75)
    plt.xlabel('X')
    plt.ylabel('Liczność')
    plt.xlim(xmin=-13,xmax=13)
    describe(data)
    
def skewness(a=0):
    data = (stats.skewnorm(a).rvs(1000)*10).astype(int)
    
    _, _, patches = plt.hist(x=data, bins='auto', color='#521422', alpha=0.7, rwidth=0.85)
    plt.grid(axis='y', alpha=0.75)
    plt.xlabel('X')
    plt.ylabel('Liczność')
    
    stat = [np.mean(data), np.median(data), stats.mode(data)[0][0]]
    cols = ['r', 'g', 'b']
    lines = [Line2D([0], [0], color=c, lw=4) for c in cols]
    
    for col, st in zip(cols, stat):
        for i, patch in enumerate(patches):
            if st < patch.get_x():
                patches[i-1].set_color(col)
                break
            if i == len(patches)-1:
                patches[i].set_color(col)
     
    plt.suptitle("współczynnik skośności="+str(round(stats.skew(data, bias=False),3)))
    plt.legend(lines,[txt +'='+str(stat[i]) for i, txt in enumerate(['Średnia', 'Mediana', 'Dominanta'])])   
    
def binary(p=0.5):
    plt.stem([0,1], [1-p, p], use_line_collection=True)
    plt.xlabel("x")
    plt.ylabel("P(X=x)")
    
def binom(n=10, p=0.5):
    plt.stem(range(n+1), stats.binom.pmf(range(n+1), n, p), use_line_collection=True)
    plt.xlabel("x")
    plt.ylabel("P(X=x)")

def uniform(a=0, b=1):
    if a >= b:
        a = b-1
    x_lim = 4
    p = 1/(b-a)
    y = [0, p, 0]
    plt.hlines(y, [-x_lim, a, b], [a, b, x_lim], color='#1f77b4')
    plt.plot([a, b], [p, p], 'o')
    plt.plot([a, b], [0, 0], 'o', fillstyle='none', color='#1f77b4')
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.xlim(-x_lim, x_lim)
    
def normal_param(mu=0, sigma=1):
    x_min = -10
    x_max = 10
    x = np.linspace(x_min, x_max, 100)
    plt.plot(x, stats.norm.pdf(x, mu, sigma))
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.xlim(x_min,x_max)
    plt.ylim(0,0.45)

def normal_s(mu=0, sigma=1, a=0, b=1):
    x_min = -10
    x_max = 10
    x_s = np.linspace(-3, 3, 100)
    x = np.linspace(x_min, x_max, 100)
    plt.plot(x_s, stats.norm.pdf(x_s, 0, 1))
    plt.plot(x, stats.norm.pdf(x, mu+a, sigma/b))
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.xlim(x_min,x_max)
    plt.ylim(0,0.45)

def clt(n = 1):
    N = 10000
    data = [np.mean(np.random.uniform(low=0, high=1, size=(n,))) for i in range(N)]
    weights = np.ones_like(data)/float(len(data))
    mu, std = stats.norm.fit(data)
    xmin, xmax = plt.xlim()
    x = np.linspace(xmin, xmax, 100)
    p = stats.norm.pdf(x, mu, std)
    plt.hist(x=data, edgecolor='black', linewidth=1.2, density=True, weights=weights)
    plt.plot(x, p, 'k', linewidth=2)
    plt.xlabel('X')
    
def monte_carlo(k=1):
    n = 10**k
    circle = plt.Circle((0, 0), 1, alpha=0.5)
    fig, ax = plt.subplots(figsize=(5,5))
    ax.add_artist(circle)
    points = np.random.uniform(size=(2,n))
    plt.scatter(*points, s=5, c='r')
    pi = np.mean(np.linalg.norm(points, axis=0)<1)*4
    plt.title(r"$n=$"+str(n)+r"  $\pi=$"+str(pi))
    plt.xlim(0,1)
    plt.ylim(0,1)
        
def conf_int(n=10, alpha=0.05):
    plt.subplots(figsize=(10,5))
    mu = 0
    sigma = 1
    ints_num = 100
    points = np.random.normal(mu, sigma, (ints_num, n))
    xs = np.mean(points, axis=1)
    z = stats.norm.ppf(1-alpha/2, mu, sigma)
    left = xs - z*sigma/np.sqrt(n)
    right = xs + z*sigma/np.sqrt(n)
    y = np.linspace(0, 100, ints_num)
    in_count = np.mean([1 if l <= mu <= r else 0 for l,r in zip(left, right)])
    colors = ['#1f77b4' if l <= mu <= r else '#ff7f50' for l,r in zip(left, right)]
    plt.vlines(y, left, right, color=colors)
    plt.plot(y, xs, '.')
    plt.axhline(y=mu, color='gray', linestyle='--')
    plt.ylim([-1.5, 1.5])
    plt.title("Proporcja przedziałów zawierających mu: "+str(in_count))
    
def fill(lower, upper, mu, sigma, col='#add8e6', ax=None):    
    x = np.linspace(lower, upper, 100)
    y = stats.norm.pdf(x,mu,sigma)
    if ax:
        ax.fill_between(x, y, color=col, alpha=0.5)
    else:
        plt.fill_between(x, y, color=col, alpha=0.5)
    
def norm_filled(mu, sigma, filled=False, ax=None):
    x_min = mu-3.5*sigma
    x_max = mu+3.5*sigma
    x = np.linspace(x_min, x_max, 100)
    if not ax:
        plt.plot(x, stats.norm.pdf(x, mu, sigma))
    else:
        ax.plot(x, stats.norm.pdf(x, mu, sigma))
    if filled:
        fill(mu-3.5*sigma, mu+3.5*sigma, mu, sigma, ax=ax)
    return x_min, x_max

def critical_region(typ_testu="prawostronny"):
    mu = 0
    sigma = 1
    x_min, x_max = norm_filled(mu, sigma)
    alpha = 0.05
    if typ_testu == "lewostronny":
        z_crit = stats.norm.ppf(alpha, mu, sigma)
        fill(x_min, z_crit, mu, sigma, "#ff4c4c")
        fill(z_crit, x_max, mu, sigma)
        plt.title(r"Test lewostronny: $C_{kr} = (-\infty, -z_{kr})$")
        plt.text(-2.25, 0.02, r"$\alpha\%$", fontsize=12)
    elif typ_testu == "prawostronny":
        z_crit = stats.norm.ppf(1-alpha, mu, sigma)
        fill(x_min, z_crit, mu, sigma)
        fill(z_crit, x_max, mu, sigma, "#ff4c4c")
        plt.title(r"Test prawostronny: $C_{kr} = (z_{kr},\infty)$")
        plt.text(1.75, 0.02, r"$\alpha\%$", fontsize=12)
    else:
        z_crit = stats.norm.ppf(alpha/2, mu, sigma)
        fill(z_crit, -1*z_crit, mu, sigma)
        fill(x_min, z_crit, mu, sigma, "#ff4c4c")
        fill(-1*z_crit, x_max, mu, sigma, "#ff4c4c")
        plt.title(r"Test: dwustronny: $C_{kr} = (-\infty, -z_{kr}) \cup  (z_{kr},\infty)$")
        plt.text(2, 0.005, r"$\alpha/2\%$", fontsize=10)
        plt.text(-2.5, 0.005, r"$\alpha/2\%$", fontsize=10)

def p_value(x_bar=106):
    fig, axes = plt.subplots(nrows=1, ncols=2, figsize=(12,5))
    mu = 0
    sigma = 1
    alpha = 0.05
    z_crit = stats.norm.ppf(1-alpha, mu, sigma)
    for ax in axes:
        norm_filled(mu, sigma, True, ax)
    fill(z_crit, 3.5, mu, sigma, "#ff4c4c", axes[0])
    x_bar = (x_bar-100)/(15/np.sqrt(30))
    axes[0].vlines(x=x_bar, ymin=0, ymax=stats.norm.pdf(x_bar, mu, sigma))
    axes[0].set_title(r"z="+str(round(x_bar,2)))
    axes[1].vlines(x=x_bar, ymin=0, ymax=stats.norm.pdf(x_bar, mu, sigma))
    fill(x_bar, 3.5, mu, sigma, "#ffa500", axes[1])
    p_val = 1-stats.norm.cdf(x_bar, mu, sigma)
    axes[1].set_title(r"p-wartość="+str(round(p_val,4)))

def chi_sq(k=1):
    fig, axes = plt.subplots(figsize=(10,5))
    x = np.linspace(0, 80, 1000)
    chi2 = stats.chi2.pdf(x, k)
    plt.plot(x, chi2)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.title(r"$\chi^2($k="+str(k)+"$)$")
    plt.xlim(0,80)

def t_dist(k=1):
    fig, axes = plt.subplots(figsize=(10,5))
    x = np.linspace(-3.25, 3.25, 1000)
    t = stats.t.pdf(x, k)
    norm = stats.norm.pdf(x)
    plt.plot(x, t, label="t("+str(k)+")")
    plt.plot(x, norm, c='k', linestyle="--", label="N(0,1)")
    quant = -1.64
    plt.vlines(x=quant, ymin=0, ymax=stats.t.pdf(quant, k), color="#ff4c4c", alpha=0.5)
    x2 = np.linspace(-3.25, -1.64, 1000)
    t2 = stats.t.pdf(x2, k)
    plt.fill_between(x2, t2, color="#ff4c4c", alpha=0.5)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.title(r"$P(T<"+str(quant)+")="+str(round(stats.t.cdf(quant, k),3))+"$")
    plt.xlim(-3.25,3.25)
    plt.ylim(0, 0.42)
    plt.legend()
    plt.grid()
    
def alpha_beta(alpha=0.05, mu1=107, n=30):
    mu0 = 100
    sigma = 15/np.sqrt(n)
    xs = [np.linspace(m-3.5*sigma, m+3.5*sigma, 100) for m in [mu0, mu1]]
    for x, m in zip(xs,[mu0, mu1]):
        plt.plot(x, stats.norm.pdf(x, m, sigma))
    z_crit = stats.norm.ppf(1-alpha, mu0, sigma)
    fill(z_crit, mu0+3.5*sigma, mu0, sigma, "#ff4c4c")
    fill(mu1-3.5*sigma, z_crit, mu1, sigma, "#98fb98")
    plt.gca().set_ylim(bottom=0)
    beta = stats.norm.cdf(z_crit, mu1, sigma)
    plt.title(r"$\beta=$"+str(round(beta,2)))
    a = mpatches.Patch(color='#ff4c4c', label=r'$\alpha$', alpha=0.5)
    b = mpatches.Patch(color='#98fb98', label=r'$\beta$', alpha=0.5)
    plt.legend(handles=[a, b])
    
def norm(n=30, x_bar=98):
    mu = 100
    sigma = 15/np.sqrt(n)
    norm_plain(mu, sigma)
    quant = -1.64*sigma+mu
    plt.vlines(x=x_bar, ymin=0, ymax=stats.norm.pdf(x_bar, mu, sigma), color="k", alpha=0.5)
    x2 = np.linspace(mu-3.25*sigma, quant, 1000)
    norm2 = stats.norm.pdf(x2, mu, sigma)
    plt.fill_between(x2, norm2, color="#ff4c4c", alpha=0.5)
    plt.xlim(91, 109)
    
def norm_plain(mu, sigma):
    fig, axes = plt.subplots(figsize=(10,5))
    x = np.linspace(mu-3.25*sigma, mu+3.25*sigma, 1000)
    norm = stats.norm.pdf(x, mu, sigma)
    plt.plot(x, norm, mu, sigma)
    plt.xlabel("x")
    plt.ylabel("f(x)")
    plt.xlim(mu-3.25*sigma, mu+3.25*sigma)
    ymax = stats.norm.pdf(mu, mu, sigma)
    ymax += 0.025*ymax
    plt.ylim(0, ymax)
    plt.grid()
    
def conf_int_test(x_bar=98):
    n = 30
    mu = 100
    sigma = 15/np.sqrt(n)
    norm_plain(mu, sigma)
    alpha = 0.05
    quants = [stats.norm.ppf(a, mu, sigma) for a in [alpha/2, 1-alpha/2]]
    for start, end in zip([mu-3.25*sigma, quants[1]], [quants[0], mu+3.25*sigma]):
        x2 = np.linspace(start, end, 1000)
        norm2 = stats.norm.pdf(x2, mu, sigma)
        plt.fill_between(x2, norm2, color="#ff4c4c", alpha=0.5)
    quant = stats.norm.ppf(1-alpha/2)
    delta = quant*sigma
    lower, upper = x_bar-delta, x_bar+delta
    plt.scatter(x_bar, 0.001, c='r', zorder=2)
    plt.hlines(y=0.001, xmin=lower, xmax=upper, color='r')
    plt.vlines(x=lower, ymin=0, ymax=0.005, color='r')
    plt.vlines(x=upper, ymin=0, ymax=0.005, color='r')
    
def corr_examples(przykład="1", typ='kowariancja', pokaż=False):
    x = np.linspace(1, 10, 100)
    y = -x**3 + np.random.normal(0,100,100) if przykład=="1" else np.cos(x) + np.random.normal(0,0.1,100)
    plt.scatter(x, y)
    plt.xlabel("x")
    plt.ylabel("y")
    if pokaż:
        if typ=="korelacja":
            plt.title("r="+str(round(np.corrcoef(x, y)[0][1],3)))
        else:
            plt.title("c="+str(round(np.cov(x, y)[0][1],3)))

def regression(nachylenie=5, sd=0, pokaż=False):
    fig, axes = plt.subplots(figsize=(7,7))
    x = np.linspace(0, 1, 100)
    yt = 1 + nachylenie * x
    plt.plot(x, yt)
    y = yt+ np.random.normal(0,sd,100)
    plt.scatter(x, y)
    my = np.mean(y)
    ssr = sum((yt-my)**2)
    sse = sum((y-yt)**2)
    sst = ssr+sse
    plt.ylim(-30,30)
    plt.xlabel("x")
    plt.ylabel("y")
    
    if pokaż:
       plt.title("SST="+str(round(sst,2))+"   SSR="+str(round(ssr, 2))+"   SSE="+str(round(sse,2)))
    
def spearman_pearson(przykład="1", pokaż=False):
    fig, axes = plt.subplots(figsize=(5,5))
    n = 50
    x = np.linspace(0, 1, n)
    y = x 
    if przykład=="2":
        y = x**5
    elif przykład=="3":
        y = x + np.random.normal(0, 0.2, n)
        x[-5:] = np.linspace(2, 3, 5)
    elif przykład=="4":
        y = x + np.random.normal(0, 1, n)
    plt.scatter(x, y)
    plt.xlabel("x")
    plt.ylabel("y")
    r = round(stats.pearsonr(x,y)[0],2)
    rs = round(stats.spearmanr(x,y)[0],2)
    if pokaż:
        plt.title(r"$r_{Pearson}="+str(r)+"$   $r_{Spearman}="+str(rs)+"$")
    plt.grid()

### Spis treści
<a href='#r1'>Wstępna analiza danych</a>:
- <a href='#1.1'>Przemnożenie/dodanie stałej do danych a wskaźniki położenia i rozproszeniah</a>
- <a href='#1.2'>Skośność a średnia, mediana i dominanta w rozkładzie unimodalnym</a>

<a href='#r2'>Od modelu probabilistycznego do wnioskowania statystycznego</a>:
- <a href='#2.1'>Rozkład dwupunktowy (zero-jedynkowy)</a>
- <a href='#2.2'>Rozkład dwumianowy</a>
- <a href='#2.3'>Rozkład jednostajny ciągły</a>
- <a href='#2.4'>Rozkład normalny</a>
- <a href='#2.5'>Standaryzacja rozkładu normalnego</a>
- <a href='#2.6'>Centralne twierdzenie graniczne</a>

<a href='#r3'>Wnioskowanie statystyczne</a>:
- <a href='#3.1'>Przedział ufności</a>
- <a href='#3.2'>Zbiór krytyczny</a>
- <a href='#3.3'>P-wartość (test prawostronny)</a>
- <a href='#3.4'>Rozkład $\chi^2$</a>
- <a href='#3.5'>Rozkład t a rozkład normalny</a>
- <a href='#3.6'>Błędy I i II rodzaju</a>
- <a href='#3.7'>Test statystyczny a przedział ufności</a>

<a href='#r4'>Analiza zależności zmiennych ilościowych</a>:
- <a href='#4.1'>Kowariancja i korelacja</a>
- <a href='#4.2'>Rozkład całkowitej zmienności zmiennej objaśnianej</a>

<a href='#r8'>Metoda Monte Carlo</a>:
- <a href='#8.1'>Zastosowanie metody Monte Carlo do szacowania wartości liczby Pi</a>

<a href='#r9'>Metody Rangowe</a>:
- <a href='#9.1'>Współczynnik korelacji Spearmana a Pearsona</a>

### <a id='r1'></a>Wstępna analiza danych

#### <a id='1.1'></a>Przemnożenie/dodanie stałej do danych a wskaźniki położenia i rozproszenia

Poniższy wykres przedstawia 1000 wartości wylosowanych z rozkładu normalnego o wartości oczekiwanej 1 i odchyleniu standardowym 1. Nad wykresem przestawione są liczność zbioru danych oraz wartości miar położenia i rozproszenia. Suwaki pozwalają na przemnożenie każdej obserwacji ze zbioru danych przez stałą (*a*) oraz dodanie do każdej obserwacji stałej (*b*).  

In [3]:
data = np.random.normal(loc=1, scale=1, size=1000)   
interact(histogram, data=fixed(data), a=(-3,3,1), b=(-10,10,1))

interactive(children=(IntSlider(value=1, description='a', max=3, min=-3), IntSlider(value=0, description='b', …

<function __main__.histogram(data, a=1, b=0)>

#### <a id='1.2'></a>Skośność a średnia, mediana i dominanta w rozkładzie unimodalnym

Poniższy wykres przedstawia histogram utworzony dla 1000 losowo wygenerowanych wartości. Na czerwono oznaczono słupek, w którym znajduje się średnia, na zielono słupek mediany, na niebiesko słupek dominanty. Jeżeli dwie lub więcej z tych miar znalazły się w słupku oznaczony jest on tylko jednym kolorem. Nad wykresem podana jest wartość skorygowanego współczynnika skośności Fishera-Pearsona. Skośność rozkładu możesz modyfikować za pomocą parametru *a*:
- *a*>0 - rozkład prawoskośny,
- *a<0* - rozkład lewoskośny.

In [4]:
interact(skewness, a=(-20,20,2))

interactive(children=(IntSlider(value=0, description='a', max=20, min=-20, step=2), Output()), _dom_classes=('…

<function __main__.skewness(a=0)>

### <a id='r2'></a>Od modelu probabilistycznego do wnioskowania statystycznego

#### <a id='2.1'></a>Rozkład dwupunktowy (zero-jedynkowy)

Parametry:
- *p* - prawdopodobieństwo sukcesu.

In [5]:
interact(binary, p=(0,1,0.1))

interactive(children=(FloatSlider(value=0.5, description='p', max=1.0), Output()), _dom_classes=('widget-inter…

<function __main__.binary(p=0.5)>

#### <a id='2.2'></a>Rozkład dwumianowy

Parametry:
- *n* - liczba doświadczeń,
- *p* - prawdopodobieństwo sukcesu.

In [6]:
interact(binom, p=(0,1,0.1), n=(5,10,1))

interactive(children=(IntSlider(value=10, description='n', max=10, min=5), FloatSlider(value=0.5, description=…

<function __main__.binom(n=10, p=0.5)>

#### <a id='2.3'></a>Rozkład jednostajny ciągły

Parametry:
- *a* - dolna granica przedziału,
- *b* - górna granica przedziału.

In [30]:
interact(uniform, a=(-3,3,1), b=(-3,3,1))

interactive(children=(IntSlider(value=0, description='a', max=3, min=-3), IntSlider(value=1, description='b', …

<function __main__.uniform(a=0, b=1)>

#### <a id='2.4'></a>Rozkład normalny

Parametry:
- *mu* - wartość oczekiwana,
- *sigma* - odchylenie standardowe.

In [8]:
interact(normal_param, mu=(-10,10,1), sigma=(1,5,1))

interactive(children=(IntSlider(value=0, description='mu', max=10, min=-10), IntSlider(value=1, description='s…

<function __main__.normal_param(mu=0, sigma=1)>

#### <a id='2.5'></a>Standaryzacja rozkładu normalnego

Niebieska krzywa przedstawia rozkład normalny o wartości oczekiwanej 0 i odchyleniu standardowym 1. Pomarańczowa krzywa przedstawia rozkład normalny o losowej wartości oczekiwanej (*m*) i odchyleniu standardowym (*sd*) wyświetlonych poniżej. Za pomocą suwaków możesz od pomarańczowego rozkładu odjąć stałą (*a*) oraz podzielić go przez stałą (*b*) aby go ustandaryzować.

In [9]:
mu = np.random.randint(-5, 5)
sigma = np.random.randint(1,5)
print("m =", mu, "sd =", sigma)
interact(normal_s, mu=fixed(mu), sigma=fixed(sigma), a = (-5,5,1), b=(1,5,1))

m = -1 sd = 3


interactive(children=(IntSlider(value=0, description='a', max=5, min=-5), IntSlider(value=1, description='b', …

<function __main__.normal_s(mu=0, sigma=1, a=0, b=1)>

#### <a id='2.6'></a>Centralne twierdzenie graniczne

Ilustracja działania centralnego twierdzenia granicznego. Poniższy wykres przedstawia histogram 10000 średnich z *n*-elementowych wektorów wylosowanych z rozkładu jednostajnego ciągłego o parametrach (0,1) oraz krzywą rozkładu normalnego. Korzystając z listy możesz zwiększyć długość wektora liczb losowych z którego liczona jest średnia.

In [10]:
interact(clt, n=[1,2,5,10,50,100])

interactive(children=(Dropdown(description='n', options=(1, 2, 5, 10, 50, 100), value=1), Output()), _dom_clas…

<function __main__.clt(n=1)>

### <a id='r3'></a>Wnioskowanie statystyczne

#### <a id='3.1'></a>Przedział ufności

Poniższy wykres przedstawia 100 przedziałów ufności na poziomie ufności 1-*alpha* skonstruowanych dla *n*-elementowych prób z rozkładu normalnego o wartości oczekiwanej 0 i odchyleniu standardowym 1. Kropką została oznaczona średnia z próby, linia pionowa to zakres przedziału, pozioma linia przerywana odpowiada wartości estymowanego parametru *mu*=0. Na pomarańczowo zostały oznaczone przedziały dla których estymowana średnia nie znalazła się w przedziale. Za pomocą suwaków możesz zmieniać parametry *n* i *alpha*.

In [11]:
interact(conf_int, n=(10, 1000, 100), alpha=(0.01, 0.5, 0.05))

interactive(children=(IntSlider(value=10, description='n', max=1000, min=10, step=100), FloatSlider(value=0.05…

<function __main__.conf_int(n=10, alpha=0.05)>

#### <a id='3.2'></a>Zbiór krytyczny

Obszar oznaczony na różowo przedstawia zbiór krytyczny dla wybranego rodzaju testu na poziomie istotności *alpha*.

In [12]:
interact(critical_region, typ_testu=["lewostronny", "prawostronny", "dwustronny"])

interactive(children=(Dropdown(description='typ_testu', index=1, options=('lewostronny', 'prawostronny', 'dwus…

<function __main__.critical_region(typ_testu='prawostronny')>

#### <a id='3.3'></a>P-wartość (test prawostronny)

*X_bar* to średnia z próby. Czarna pionowa linia na wykresie po lewej stronie przedstawia jej wartość po standaryzacji (*z*), obszar oznaczony na różowo to zbiór krytyczny. Na wykresie po prawej stronie obszar zaznaczony na żółto odpowiada prawdopodobieństwu zaobserwowania wartości równej lub wyższej *x_bar* przy założeniu prawdziwości hipotezy zerowej (p-wartość).

In [13]:
interact(p_value, x_bar=(100, 110, 1))

interactive(children=(IntSlider(value=106, description='x_bar', max=110, min=100), Output()), _dom_classes=('w…

<function __main__.p_value(x_bar=106)>

#### <a id='3.4'></a>Rozkład $\chi^2$

In [14]:
interact(chi_sq, k=(1,40,1))

interactive(children=(IntSlider(value=1, description='k', max=40, min=1), Output()), _dom_classes=('widget-int…

<function __main__.chi_sq(k=1)>

#### <a id='3.5'></a>Rozkład t a rozkład normalny

Czarna przerywana krzywa przedstawia rozkład normalny o wartości oczekiwanej 0 i odchyleniu standardowym 1. Niebieska krzywa przedstawia rozkład t o *k* stopniach swobody. Obszar zaznaczony na różowo odpowiada prawdopodobieństwu, że zmienna losowa *T* przyjmie wartość mniejszą niż 1.64. Za pomocą suwaka możesz zmieniać wartość parametru *k*.

In [15]:
interact(t_dist, k=(1,100,1))

interactive(children=(IntSlider(value=1, description='k', min=1), Output()), _dom_classes=('widget-interact',)…

<function __main__.t_dist(k=1)>

#### <a id='3.6'></a>Błędy I i II rodzaju

Niebieska krzywa przedstawia rozkład statystyki testowej dla założonej w hipotezie zerowej wartości *mu0*=100. Pomarańczowa krzywa przedstawia rozkład statystyki testowej dla alternatywnej wartości testowanego parametru *mu1*. Na różowo oznaczone zostało prawdopodobieństwo popełnienia błędu I rodzaju *alpha* (test prawostronny). Na zielono oznaczone zostało prawdopodobieństwo popełnienia błędu II rodzaju *Beta*. Za pomocą suwaków możesz zmieniać poziom istotności *alpha*, wartość parametru *mu1* i liczność próby *n*. 

In [16]:
interact(alpha_beta, alpha=(0.01, 0.25, 0.01), mu1=(100, 110, 1), n=(30, 500, 5))

interactive(children=(FloatSlider(value=0.05, description='alpha', max=0.25, min=0.01, step=0.01), IntSlider(v…

<function __main__.alpha_beta(alpha=0.05, mu1=107, n=30)>

#### <a id='3.7'></a>Test statystyczny a przedział ufności

Poniższy wykres ilustruje zależność pomiędzy dwustronnym testem statystycznym dla jednej populacji na poziomie istotności *alpha* a przedziałem ufności dla wartości średniej rozkładu normalnego na poziomie ufności 1-*alpha*. Wartość parametru *mu* założona w hipotezie zerowej to 100. Rozmiar próby wynosi 30. Na różowo został oznaczony zbiór krytyczny w teście dwustronnym na poziomie istotności *alpha*. Oznaczona czerwoną kropką *x_bar* to wartość statystyki z próby. Czerwone "wąsy" oznaczają przedział ufności skonstruowany w oparciu o *x_bar*. Za pomocą suwaka możesz zmieniać wartość *x_bar*. Aby odrzucić hipotezę zerową wartość statystyki z próby musi znaleźć się w obszarze krytycznym. Zauważ, że wówczas założone w hipotezie zerowej *mu*=100 znajdzie się poza przedziałem ufności.      

In [17]:
interact(conf_int_test, x_bar=(90, 110, 1))

interactive(children=(IntSlider(value=98, description='x_bar', max=110, min=90), Output()), _dom_classes=('wid…

<function __main__.conf_int_test(x_bar=98)>

### <a id='r4'></a>Analiza zależności zmiennych ilościowych

#### <a id='4.1'></a>Kowariancja i korelacja

Poniższa wizualizacja przedstawia wykres rozrzutu dwóch zmiennych *x* i *y* dla których policzono próbkowy współczynnik kowariancji i współczynnik korelacji Pearsona. Zastanów się nad znakiem i wartością tych współczynników, a następie je wyświetl, korzystając z opcji "pokaż".

In [18]:
interact(corr_examples, typ=["kowariancja", "korelacja"], przykład=["1", "2"])

interactive(children=(Dropdown(description='przykład', options=('1', '2'), value='1'), Dropdown(description='t…

<function __main__.corr_examples(przykład='1', typ='kowariancja', pokaż=False)>

#### <a id='4.2'></a>Rozkład całkowitej zmienności zmiennej objaśnianej

Poniższa wizualizacja przedstawia wykres rozrzutu dwóch zmiennych *x* i *y* oraz prostą regresji. Za pomocą parametru *sd* można zwiększać rozrzut danych wokół prostej regresji. Zastanów się jak wraz ze wzrostem rozrzutu będzie zmieniać się proporcja regresyjnej sumy kwadratów (*SSR*) do sumy kwadratów błędów (*SSE*) a następnie je wyświetl korzystając z opcji "pokaż".

In [19]:
interact(regression, nachylenie=fixed(5), sd=(0,10,1))

interactive(children=(IntSlider(value=0, description='sd', max=10), Checkbox(value=False, description='pokaż')…

<function __main__.regression(nachylenie=5, sd=0, pokaż=False)>

### <a id='r8'></a>Metoda Monte Carlo

#### <a id='8.1'></a>Zastosowanie metody Monte Carlo do szacowania wartości liczby Pi

Poniższy wykres ilustruje zastosowanie metody Monte Carlo do szacowania wartości liczby Pi. Ćwierć koła o promieniu 1 jest wpisane w kwadrat o boku 1. Współrzędne 10^*k* punktów są losowane z rozkładu jednostajnego. Proporcja punktów które znalazły się wewnątrz koła do wszystkich punktów przybliża proporcję tych dwóch figur, Pi/4. Pomnożenie tej wartości przez 4 daje oszacowanie wartości liczby Pi.

In [20]:
interact(monte_carlo, k=(1,4,1))

interactive(children=(IntSlider(value=1, description='k', max=4, min=1), Output()), _dom_classes=('widget-inte…

<function __main__.monte_carlo(k=1)>

### <a id='r9'></a>Metody Rangowe

#### <a id='9.1'></a>Współczynnik korelacji Spearmana a Pearsona

Poniższa wizualizacja przedstawia kilka par zmiennych ilościowych. Spróbuj oszacować wartości współczynników korelacji Spearmana i Pearsona dla każdego z przykładów, a następnie je wyświetl korzystając z opcji "pokaż". 

In [21]:
interact(spearman_pearson, przykład=["1","2","3","4"])

interactive(children=(Dropdown(description='przykład', options=('1', '2', '3', '4'), value='1'), Checkbox(valu…

<function __main__.spearman_pearson(przykład='1', pokaż=False)>